<a href="https://colab.research.google.com/github/AbdassalamAhmad/Gender-Classification/blob/main/Gender_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gender Classification using Xception model

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array


# Getting Dataset

In [3]:
!git clone https://github.com/laxmimerit/male-female-face-dataset.git

fatal: destination path 'male-female-face-dataset' already exists and is not an empty directory.


### Declaring Variables

In [4]:
batch_size = 128
size = 224 #width and height
target_size = (size, size, 3)

epochs = 2
lr = 1e-3


# Image Data Generator (without augmentation)

In [5]:
train_datagen = ImageDataGenerator(rescale=1./255)

In [6]:
val_datagen = ImageDataGenerator(rescale=1./255)

In [7]:
train_dataset = train_datagen.flow_from_directory(
    directory = '/content/male-female-face-dataset/Training',
    target_size = (size, size),
    batch_size = batch_size,
    class_mode = 'binary'
)

Found 47009 images belonging to 2 classes.


In [8]:
val_dataset = val_datagen.flow_from_directory(
    directory = '/content/male-female-face-dataset/Validation',
    target_size = (size, size),
    batch_size = batch_size,
    class_mode = 'binary'
)

Found 11649 images belonging to 2 classes.


# Building Xception Model

In [9]:
#base
model = Sequential()
model.add(Xception(include_top=False, pooling='avg', weights='imagenet', input_shape=target_size))
model.add(Flatten())
model.add(BatchNormalization())
#head
model.add(Dense(2048, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())

model.add(Dense(1, activation='sigmoid'))

model.layers[0].trainable = False

83697664/83683744 [==============================] - 1s 0us/step


In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 2048)              20861480  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 batch_normalization_4 (Batc  (None, 2048)             8192      
 hNormalization)                                                 
                                                                 
 dense (Dense)               (None, 2048)              4196352   
                                                                 
 batch_normalization_5 (Batc  (None, 2048)             8192      
 hNormalization)                                                 
                                                                 
 dense_1 (Dense)             (None, 1024)              2

In [11]:
model.compile(optimizer= 'Adam', loss = 'binary_crossentropy', metrics= 'accuracy')

In [12]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('xception_v1_{epoch:02d}_{val_accuracy:.3f}.h5',
                                             save_best_only = True,
                                             monitor= 'val_accuracy',
                                             mode = 'max')

In [14]:
history = model.fit(train_dataset, epochs= epochs, validation_data= val_dataset,
                    steps_per_epoch= len(train_dataset.filenames)//batch_size,
                    validation_steps= len(val_dataset.filenames)//batch_size,
                    callbacks= [checkpoint]
                    )

Epoch 1/2
367/367 [==============================] - 497s 1s/step - loss: 0.1904 - accuracy: 0.9273 - val_loss: 0.1701 - val_accuracy: 0.9325
Epoch 2/2
367/367 [==============================] - 494s 1s/step - loss: 0.1625 - accuracy: 0.9379 - val_loss: 0.2020 - val_accuracy: 0.9200


# Testing the Model

In [15]:
img_path = '/content/male-female-face-dataset/Validation/male/063429.jpg.jpg'
#img_path = '/content/WIN_20220217_06_43_27_Pro.jpg'
#img_path = '/content/male-female-face-dataset/Validation/female/112944.jpg.jpg'
#plt.imshow(img)

In [16]:
def predict_img(img_path):
  #preprocess Image
  img = load_img(img_path, target_size=(target_size))
  img = img_to_array(img)
  img = img/255.0
  img=img.reshape(1, size, size, 3)

  #Prediction
  pred = model.predict(img)
  pred = pred[0][0]  
  if pred>=0.5:
    result = "Male, Confidence is {:.2f}".format(pred)
    return result
  else:
    pred = 1-pred
    result = "Female, Confidence is {:.2f}".format(pred)
    return result
predict_img(img_path)

'Male, Confidence is 0.98'

# Training with Data Augmentation

In [17]:
train_datagen = ImageDataGenerator(horizontal_flip=True,
                                   width_shift_range=0.4,
                                   height_shift_range=0.4,
                                   zoom_range=0.3,
                                   rotation_range=20,
                                   rescale=1./255)

train_dataset = train_datagen.flow_from_directory(
    directory = '/content/male-female-face-dataset/Training',
    target_size = (size, size),
    batch_size = batch_size,
    class_mode = 'binary'
)

Found 47009 images belonging to 2 classes.


In [ ]:
epochs = 10
checkpoint = tf.keras.callbacks.ModelCheckpoint('xception_v2_{epoch:02d}_{val_accuracy:.3f}.h5',
                                             save_best_only = True,
                                             monitor= 'val_accuracy',
                                             mode = 'max')

history = model.fit(train_dataset, epochs= epochs, validation_data= val_dataset,
                    steps_per_epoch= len(train_dataset.filenames)//batch_size,
                    validation_steps= len(val_dataset.filenames)//batch_size,
                    callbacks= [checkpoint]
                    )

Epoch 1/10
367/367 [==============================] - 786s 2s/step - loss: 0.3279 - accuracy: 0.8594 - val_loss: 0.2020 - val_accuracy: 0.9172
Epoch 2/10
367/367 [==============================] - 791s 2s/step - loss: 0.3028 - accuracy: 0.8732 - val_loss: 0.2152 - val_accuracy: 0.9126
Epoch 3/10
367/367 [==============================] - 792s 2s/step - loss: 0.2930 - accuracy: 0.8766 - val_loss: 0.1903 - val_accuracy: 0.9253
Epoch 4/10
367/367 [==============================] - 778s 2s/step - loss: 0.2845 - accuracy: 0.8830 - val_loss: 0.1905 - val_accuracy: 0.9222
Epoch 5/10
367/367 [==============================] - 786s 2s/step - loss: 0.2809 - accuracy: 0.8831 - val_loss: 0.1844 - val_accuracy: 0.9257
Epoch 6/10
367/367 [==============================] - 798s 2s/step - loss: 0.2781 - accuracy: 0.8842 - val_loss: 0.1861 - val_accuracy: 0.9234
Epoch 7/10
367/367 [==============================] - 796s 2s/step - loss: 0.2752 - accuracy: 0.8851 - val_loss: 0.1854 - val_accuracy: 0.9263